<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_57.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4

In [ ]:
# !unzip "/content/joint_representations.zip"

In [ ]:
# !gdown https://drive.google.com/uc?id=1wgl3QGXZ4m2aLg3T-1TDXQqSP31RuXgL

In [ ]:
# !unzip "/content/hateful_train+test_unseen.zip"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Validation


In [2]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
model = ViTForImageClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000")

loading configuration file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "0",
    "1": "1"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "problem_type": "single_label_classification",
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.33.1"
}

loading weights file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/pytorch_model.bin
All model checkpoint weights were used when ini

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
for i in range(150,500):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [5]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [6]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/hateful_ViT1/test", split="train")

Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]

In [7]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [8]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.5

In [9]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

Counter({0: 1250, 1: 750})


In [10]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

{0: [], 1: []}


In [11]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [12]:
# selected_indices

In [13]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [14]:
# selected_indices

In [15]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [16]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1000
})

In [17]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

Resolving data files:   0%|          | 0/4000 [00:00<?, ?it/s]

In [18]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [19]:
ds_imgs = ds_joint.filter(filter_funtion)

In [20]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [21]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [22]:
all_indices = range(len(ds_imgs))
print(all_indices)

range(0, 2000)


In [23]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [24]:
# remaining_indices

In [25]:
missing_ds = ds_imgs.select(remaining_indices)

In [26]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1000
})

In [27]:
# Combine Both Datasets

In [28]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [29]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

# Pre-Processing and Validation Loop

In [30]:
labels = ds.features["label"]
labels

ClassLabel(names=['0', '1'], id=None)

In [31]:
labels.int2str(ds[532]["label"])

'0'

In [32]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [33]:
dataset = ds.with_transform(transform)

In [34]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [35]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [36]:
labels = ds.features["label"].names
labels

['0', '1']

In [37]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [38]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [39]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [40]:
batch_size = 32

In [41]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [42]:
model = model.to(device)
model.eval()
predictions, labels = [], []
valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    loss = outputs.loss
    valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [44]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("AUROC_score: ", AUROC_score)

Accuracy:  0.735
precision:  0.7317198167623603
f1_score:  0.7327644692408065
AUROC_score:  0.7098666666666666


In [45]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[1013  237]
 [ 293  457]]
